In [1]:
import os
import sys
import numpy as np

import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom
import lsst.afw.math as afwMath
import lsst.afw.cameraGeom as cameraGeom
import lsst.afw.cameraGeom.utils as cgUtils
import lsst.afw.display as afwDisplay

import matplotlib.pyplot as plt

%matplotlib ipympl
plt.rcParams['figure.figsize'] = [7, 6]

In [2]:
# For some reason, to get interactive plots, we need to run this command twice
%matplotlib ipympl

In [3]:
afwDisplay.setDefaultBackend("matplotlib")

In [4]:
butler = dafPersist.Butler("/datasets/hsc/repo" + "/rerun/RC/w_2019_14/DM-18300-jointcal")

dataId = dict(visit=436, ccd=43)

In [5]:
camera = butler.get("camera")

if True:
    #plt.close('all')
    cgUtils.plotFocalPlane(camera, useIds=True)

FigureCanvasNbAgg()

In [6]:
import lsst.obs.base.yamlCamera as yamlCamera

outDir = "/home/rlupton/LSST/obs/subaru/policy"
ycamera = yamlCamera.makeCamera(os.path.join(outDir, "hsc.yaml"))

if False:
    #plt.close('all')
    cgUtils.plotFocalPlane(ycamera, useIds=True)

In [9]:
disp = afwDisplay.Display(2, reopenPlot=True)
disp.scale('linear', 'zscale')

dtype = "raw"
callback = cgUtils.rawCallback if dtype == "raw" else None

if True:
    if True:
        _camera, cameraType = camera, "master"
    else:
        _camera, cameraType = ycamera, "yaml"

    mos = cgUtils.showCamera(_camera, 
                         cgUtils.ButlerImage(butler, dtype, visit=dataId["visit"], callback=callback, verbose=True), 
                         #detectorNameList=[49, 50, 57, 58, 65, 66][0:], 
                         #detectorNameList=["1_05", "1_06", "1_09", "1_10", "1_13", "1_14"],
                         #detectorNameList=["1_07", "1_03", "1_35", "1_06", "1_02", "1_34", "1_41"],
                         binSize=4, display=disp, title=("%s %s visit=%d" % (cameraType, dtype, dataId["visit"])), overlay=True)
else:
    disp.mtv(mos)
    
disp.show_colorbar()

FigureCanvasNbAgg()

RHL 1_53 False
RHL 1_54 False
RHL 1_55 False
RHL 1_56 False
RHL 1_42 False
RHL 1_43 False
RHL 1_44 False
RHL 1_45 False
RHL 1_46 False
RHL 1_47 False
RHL 1_36 False
RHL 1_37 False
RHL 1_38 False
RHL 1_39 False
RHL 1_40 False
RHL 1_41 False
RHL 0_30 False
RHL 0_29 False
RHL 0_28 False
RHL 1_32 False
RHL 1_33 False
RHL 1_34 False
RHL 0_27 False
RHL 0_26 False
RHL 0_25 False
RHL 0_24 False
RHL 1_00 False
RHL 1_01 False
RHL 1_02 False
RHL 1_03 False
RHL 0_23 False
RHL 0_22 False
RHL 0_21 False
RHL 0_20 False
RHL 1_04 False
RHL 1_05 False
RHL 1_06 False
RHL 1_07 False
RHL 0_19 False
RHL 0_18 False
RHL 0_17 False
RHL 0_16 False
RHL 1_08 False
RHL 1_09 False
RHL 1_10 False
RHL 1_11 False
RHL 0_15 False
RHL 0_14 False
RHL 0_13 False
RHL 0_12 False
RHL 1_12 False
RHL 1_13 False
RHL 1_14 False
RHL 1_15 False
RHL 0_11 False
RHL 0_10 False
RHL 0_09 False
RHL 0_08 False
RHL 1_16 False
RHL 1_17 False
RHL 1_18 False
RHL 1_19 False
RHL 0_07 False
RHL 0_06 False
RHL 0_05 False
RHL 0_04 False
RHL 1_20 F

In [11]:
ccd = 29 if True else 70
detectorName = camera[ccd].getName()
exp = butler.get('flat', dataId, ccd=ccd)

disp = afwDisplay.Display(3, reopenPlot=True)
disp.scale('asinh', 'zscale', Q=0.5)

disp.mtv(exp.image, title=detectorName)

FigureCanvasNbAgg()

In [20]:
raw.getInfo().getFilter().getName()

'y'

In [12]:
raw = butler.get('raw', dataId)
calexp = butler.get('calexp', dataId)

In [9]:
disp = afwDisplay.Display(3, reopenPlot=True)
disp.scale('asinh', 'zscale', Q=0.5)

disp.mtv(calexp.image, title=dataId)
#cgUtils.overlayCcdBoxes(ycamera[calexp.getDetector().getName()], display=disp)

FigureCanvasNbAgg()

In [10]:
for a in camera[calexp.getDetector().getName()]:
    print(a.getName(), a.getReadoutCorner(), a.getRawFlipX(), a.getRawFlipY())

0 ReadoutCorner.LL False False
1 ReadoutCorner.LR False False
2 ReadoutCorner.LL False False
3 ReadoutCorner.LR False False


In [11]:
for a in ycamera[calexp.getDetector().getName()]:
    print(a.getName(), a.getReadoutCorner(), a.getRawFlipX(), a.getRawFlipY())

A0 ReadoutCorner.LL False False
A1 ReadoutCorner.LR False False
A2 ReadoutCorner.LL False False
A3 ReadoutCorner.LR False False


In [12]:
def getSubCCD(detector):
    return detector.getName()[2:]   # i.e. 32 if detector.getName() == "0_32" or "1_32"

def getRaftName(detector):
    return detector.getName()[0]    # i.e. 0  if detector.getName() == "0_26"

def printRafts(camera, fd, plateScale=1, cameraCenter=afwGeom.PointD(0, 0)):
    """Print the second part of the cameraHeader.yaml file"""
    print("""
#
# Layout of CCDs within a raft
#
RAFT_HAMAMATSU:\
""", file=fd)
    print("  ccds :", file=fd)

    nominalPosition = {}
    for i, det in enumerate(sorted(camera, key=lambda a : a.getName())):
        sname = getSubCCD(det)

        xc, yc = det.getCenter(cameraGeom.FOCAL_PLANE) - cameraCenter
        if getRaftName(det) == "0":
            odet = camera["1_%s" % sname]
            oxc, oyc = odet.getCenter(cameraGeom.FOCAL_PLANE) - cameraCenter

            xc = 0.5*(xc - oxc)
            yc = 0.5*(yc - oyc)
            xc *= plateScale; yc *= plateScale
            xc = np.round(10*xc)/10
            yc = np.round(10*yc)/10

            nominalPosition[sname] = (xc, yc)

            print("    '%s' : &%s_HAMAMATSU" % (sname, sname), file=fd)
            print("      << : *CCD", file=fd)
            print("      offset : [%.1f, %.1f]" % (xc, yc), file=fd)
            iyaw = np.round(det.getOrientation().getYaw().asDegrees()/90)*90
            if iyaw != 0:
                print("      yaw : %d" % (iyaw), file=fd)

            print("", file=fd)
            
    return nominalPosition

def printDetectorIds(camera, raftName, fd):
    """Print the detectorIds that go in rafts.yaml if we can't use id0"""
    print("""\
rafts :
  '%s' : 
    XXX add offset/yaw
""" % (raftName), file=fd)

    print("    detectorIds :", file=fd)
    for det in camera:
        sname = getSubCCD(det)

        if getRaftName(det) != raftName:
            continue

        print("      '%s' : %s" % (sname, det.getSerial()), file=fd)

def printSingleRaft(camera, nominalPositions, raftName, fd):
    """Print an entire rafts.yaml file"""
    print("""\
'%s' : 
  detectorType : HAMAMATSU
  raftSerial : R%s
""" % (raftName, raftName), file=fd)

    print("  ccdSerials :", file=fd)
    for det in camera:
        sname = getSubCCD(det)

        if getRaftName(det) != raftName:
            continue

        #print("    '%s' : '%s_%s'" % (sname, raftName, det.getSerial()), file=fd)
        print("    '%s' : 'SER%s'" % (sname, det.getSerial()), file=fd)

    print("", file=fd)
    print("  sensorTypes :   # this should be moved to rafts.yaml and generateCamera.py should be updated", file=fd)
    for det in camera:
        sname = getSubCCD(det)

        if getRaftName(det) != raftName:
            continue

        if det.getType() != cameraGeom.DetectorType.SCIENCE:
            detType = str(det.getType()).split(".")[-1]
            print("    '%s' : %s" % (sname, detType), file=fd)

    print("", file=fd)
    print("  geometryWithinRaft:", file=fd)
    for det in camera:
        sname = getSubCCD(det)

        if getRaftName(det) != raftName:
            continue

        xc, yc = det.getCenter(cameraGeom.FOCAL_PLANE) - cameraCenter

        xc *= plateScale; yc *= plateScale
        
        xcn, ycn = nominalPositions[sname]
        if raftName == "0":    # rotated into lower-left of focal plane
            xc, yc = -xc, -yc

        print("    '%s' :" % (sname), file=fd)
        print("      offset : [%.4f, %.4f]" % (xc + xcn, yc + ycn), file=fd)
        print("      yaw : %.4f" % (np.fmod(det.getOrientation().getYaw().asDegrees() - 90*det.getOrientation().getNQuarter(), 90)), file=fd)

    print("", file=fd)
    print("  amplifiers:", file=fd)
    for det in camera:
        sname = getSubCCD(det)

        if getRaftName(det) != raftName:
            continue
        print("    '%s' :" % (sname), file=fd)
        for a in det:
            print("        A%s : {gain : %6.4f, readNoise : %5.2f, saturation : %.0f }" % 
                  (a.getName(), a.getGain(), a.getReadNoise(), a.getSaturation()), file=fd)        
#
# ------------------------
#

plateScale = 0.015   # mm/pixel
cameraCenter = afwGeom.PointD(0, 200)  # pixels!
raftName = "0"

with open(os.path.join(outDir, "cameraHeader-rafts.yaml"), "w") as fd:
    nominalPositions = printRafts(camera, fd, plateScale=plateScale, cameraCenter=cameraCenter)

for raftName in ["0", "1"]:
    with open(os.path.join(outDir, "rafts-%s-detectorIds.yaml" % (raftName)), "w") as fd:
        printDetectorIds(camera, raftName, fd)

    with open(os.path.join(outDir, ("%s.yaml" % (raftName))), "w") as fd:
        printSingleRaft(camera, nominalPositions, raftName, fd)

In [13]:
for det in sorted(ycamera, key=lambda a : a.getName()):
    odet = camera[det.getName()]
    for a, oa in zip(det, odet):
        print(det.getName(), a.getName(), oa.getSaturation(), a.getSaturation())

0_00 A0 44000.0 44000.0
0_00 A1 44000.0 44000.0
0_00 A2 44000.0 44000.0
0_00 A3 44000.0 44000.0
0_01 A0 51500.0 51500.0
0_01 A1 51500.0 51500.0
0_01 A2 51500.0 51500.0
0_01 A3 51500.0 51500.0
0_02 A0 50500.0 50500.0
0_02 A1 50500.0 50500.0
0_02 A2 50500.0 50500.0
0_02 A3 50500.0 50500.0
0_03 A0 47000.0 47000.0
0_03 A1 47000.0 47000.0
0_03 A2 47000.0 47000.0
0_03 A3 47000.0 47000.0
0_04 A0 43000.0 43000.0
0_04 A1 43000.0 43000.0
0_04 A2 43000.0 43000.0
0_04 A3 43000.0 43000.0
0_05 A0 49000.0 49000.0
0_05 A1 49000.0 49000.0
0_05 A2 49000.0 49000.0
0_05 A3 49000.0 49000.0
0_06 A0 45000.0 45000.0
0_06 A1 45000.0 45000.0
0_06 A2 45000.0 45000.0
0_06 A3 45000.0 45000.0
0_07 A0 50000.0 50000.0
0_07 A1 50000.0 50000.0
0_07 A2 50000.0 50000.0
0_07 A3 50000.0 50000.0
0_08 A0 51000.0 51000.0
0_08 A1 51000.0 51000.0
0_08 A2 51000.0 51000.0
0_08 A3 51000.0 51000.0
0_09 A0 50000.0 50000.0
0_09 A1 50000.0 50000.0
0_09 A2 50000.0 50000.0
0_09 A3 50000.0 50000.0
0_10 A0 55000.0 55000.0
0_10 A1 55000.0 

In [7]:
import lsst.jointcal
lsst.jointcal.__version__

'17.0.1-14-gec16405'